In [37]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [39]:
url = ('https://www.dunlopaircrafttyres.co.uk/tyres/?search=5&type=size&page=1')
response = requests.get(url)
response

<Response [200]>

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')
soup

In [ ]:
data = []
current_page = 1
proceed = True

In [ ]:
while proceed:
  url = f''
  print(f'Currently fetching page {current_page}')
  response = requests.get(url)

  # checking if website is available
  if response.status_code != 200:
    print(f'Request failed on page {current_page}')
    break

  soup = BeautifulSoup(response.text, 'html.parser')

  if soup.find('div' class_='no-results active'):
    print('No more results')
    break
  '''
  We are using the selector method, to choose the css value in the page is
  <div class="part-list"> as the main table
  <div class="row" data-url=""> as row data.
  Using the selector methods enables to use css formating ie <div.class> intsead of <div class>
  '''
  # Getting whole row data from soup
  rows_data = soup.select('div.part-list div.row[data-url]')
  # Getting specific cell  from rows_data
  cells_data = rows_data.select('div.clickable.cell')
  # Getting the data in each cell
  values = {} # Dic for final data
  for cell in cells_data:
    label = cell.find('span', class_='label')
    value = cell.find('span', class_='value')
    if label and value:
      key = label.text.replace(':','').strip()
      val = value.text.strip()
      values[key] = val # Appending to values dict

      data.append(
          {
              'Part No':values.get('Part No', ''),
              'Size':values.get('Size', ''),
              'Aircraft':values.get('Aircraft',''),
              'Position':values.get('Position','')

          }
      )

  current_page += 1
  time.sleep(2.0)







In [ ]:
df = pd.DataFrame(data)
df.to_csv('Scraped_Data.csv', index=False)
print(f'Done!. Data Saved')